<a href="https://colab.research.google.com/github/samyak197/summarization-of-law-documents/blob/main/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.2 MB/s eta 0:00:00


In [2]:
!gdown --folder https://drive.google.com/drive/folders/15-QHDFT5Zlyv6nCQMgs1igVTEi5RhOur?usp=drive_link

Retrieving folder contents
Processing file 1V3Pdo3L2J1CmgixYe0Tw9mVT6mAHe39X 04c94c6b5aa1da79e36594b0ee0995334ae97d88244aac9c844366b7bf94b3af1724494149.pdf
Processing file 1m5lP-ay1_7bZH3CgHaspMNUb8A6cTjqX 4e12e76b80ecaae4f0f61e273369ac4bdda56e914a70fcb1a4aa8d630171cec21724495000.pdf
Processing file 1zNPdfXv6gQSaTPrvSix7ytHrMvMp0-w3 6d85e5806191aafbf2c10393f960042cbe173c8721a215afb65602e81b65b0201724494884.pdf
Processing file 1-b9ieoQkE_Q1mopWG9miYbpFS5lTJyIf 07fbce92174cb5a1c5ae6be5b90f418bb57bd213c9f8da06b343bef0f66f5a361724493750.pdf
Processing file 1xq_8FVuquWNWv2PPfgT8XC8OP9P0X64a 13f58a9fd5ae38e8cc4fefd40dea07a5cefd755f937d6fdc8fc481548043c2a61724494977.pdf
Processing file 1nom5LhnFis8gQ1niSddWLX4UYfnYhxS- 632330e2ae25441a8767f3874f59c860033800f4b9e8d95213926a83b139929e1724495135.pdf
Processing file 1_0AaeRauVYjaH4n4-9O0lGYgS2xR3s8x a5012a2f5be4c20636625147339dc05cd9c89ae90c034f11e914d537b60b827b1724493963.pdf
Processing file 1J6_GDoVSi95CKeSjPD-RITGnQmA1vGqZ bda980fee73361180449

In [3]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

pdf_text = extract_text_from_pdf("/content/Prathamesh/04c94c6b5aa1da79e36594b0ee0995334ae97d88244aac9c844366b7bf94b3af1724494149.pdf")


In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")

# Tokenize the extracted text
encoded_input = tokenizer(pdf_text, return_tensors="pt", truncation=True, max_length=512)

# Get embeddings
output = model(**encoded_input)
last_hidden_state = output.last_hidden_state


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/534M [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import pipeline

def split_text(text, max_length=1024):
    # Split the text into chunks of the specified max length
    return [text[i:i + max_length] for i in range(0, len(text), max_length)]

chunks = split_text(pdf_text)

# Load the summarization pipeline and assign it to GPU if available
summarizer = pipeline("summarization", device=0)

summaries = []
for chunk in chunks:
    # Summarize each chunk
    summary_chunk = summarizer(chunk, max_length=200, min_length=50, do_sample=False)
    summaries.append(summary_chunk[0]['summary_text'])

# Combine all summaries into one
final_summary = ' '.join(summaries)

# Print the final summary
print(final_summary)

# Save the summary to a text file
with open('summary.txt', 'w') as file:
    file.write(final_summary)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Your max_length is set to 200, but your input_length is only 108. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)


 Aniruddha Bose* and Sanjay Kumar, JJ . The application for modification of a judgment or an order in a matter which stands finally concluded. The application should lie only in rare cases, where the order passed by the Supreme Court is executory in nature .  There are two Orders in the Supreme Court Rules, 2013, which permit review of a judgment or an order of the Supreme Court, Orders XLVII and XLVIII . The former Order, contained in 1024  [2024] 3 S.C.R.R., relates to ‘Review of a Judgment’ and ‘Curative Petition’ by taking out a Miscellaneous Application, the application was registered as a miscellaneous application .  The application was projected as an application for clarification, though the same was registered as a miscellaneous application . The reliefs asked for in this application do not refer to any clarification . Review petitions filed against  this judgment by the Rajasthan Discoms stood dismissed on .02.03.2021 .  An important  question of law has arisen as regards jur

In [5]:
print(len(pdf_text))

47503


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the model and tokenizer
model_name = "facebook/bart-large-cnn"  # Replace with your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Get the model's maximum input length (token limit)
max_input_length = tokenizer.model_max_length
print(f"Max token limit for {model_name}: {max_input_length} tokens")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Max token limit for facebook/bart-large-cnn: 1000000000000000019884624838656 tokens


In [12]:
import os

# Ensure a directory exists for saving summaries
output_dir = "summarization_outputs"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def get_optimal_max_length(input_text, input_ratio=0.5, min_length=30, max_length=150):
    """
    Determine the best max_length for the summarizer based on the input text length.
    Adjusts the max_length based on a ratio of the input length.
    """
    input_length = len(input_text.split())  # Length of the input in words
    # Set max_length as a percentage of the input length, bounded by min/max values
    optimal_max_length = min(max(int(input_length * input_ratio), min_length), max_length)
    return optimal_max_length

def fragment_text(text, max_chunk_length=1024):
    """
    Fragment a long text into smaller chunks to feed into the summarization model.
    Each chunk should have a max length of `max_chunk_length` tokens.
    """
    sentences = text.split('. ')
    current_chunk = []
    current_length = 0
    chunks = []

    for sentence in sentences:
        sentence_length = len(sentence)
        if current_length + sentence_length <= max_chunk_length:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            chunks.append(". ".join(current_chunk) + ".")
            current_chunk = [sentence]
            current_length = sentence_length

    if current_chunk:
        chunks.append(". ".join(current_chunk) + ".")

    return chunks

def save_summary_to_file(summary_text, ratio):
    """
    Save the full summarized text into a file, with proper naming.
    """
    file_name = f"summary_ratio_{int(ratio*100)}.txt"
    file_path = os.path.join(output_dir, file_name)
    with open(file_path, 'w') as f:
        f.write(summary_text)
    print(f"Summary saved to {file_path}")

# Fragment your pdf_text
text_chunks = fragment_text(pdf_text, max_chunk_length=1024)

# Summarizer pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn",device=0)

# Dictionary to store complete summaries for each ratio
complete_summaries = {0.3: [], 0.4: [], 0.5: []}

# Summarize each chunk with different ratios and store them
for chunk_id, chunk in enumerate(text_chunks):
    for ratio in complete_summaries.keys():  # Test different summarization ratios
        optimal_max_len = get_optimal_max_length(chunk, input_ratio=ratio)
        summary = summarizer(chunk, max_length=optimal_max_len, min_length=30, do_sample=False)
        summary_text = summary[0]['summary_text']

        # Append the summary text for this chunk to the complete summary for the ratio
        complete_summaries[ratio].append(summary_text)

# Save complete summaries for each ratio as text files
for ratio, summary_list in complete_summaries.items():
    full_summary = "\n".join(summary_list)  # Combine all chunks into a full summary
    save_summary_to_file(full_summary, ratio)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Summary saved to summarization_outputs/summary_ratio_30.txt
Summary saved to summarization_outputs/summary_ratio_40.txt
Summary saved to summarization_outputs/summary_ratio_50.txt


In [13]:
import os

# Directory where the text files are stored
output_dir = "summarization_outputs"

def clean_extra_spaces(file_path):
    """
    Remove all occurrences of multiple spaces from the file content.
    """
    with open(file_path, 'r') as file:
        content = file.read()

    # Replace multiple spaces with a single space
    cleaned_content = content.replace("                ", " ")

    # Write the cleaned content back to the file
    with open(file_path, 'w') as file:
        file.write(cleaned_content)

    print(f"Cleaned extra spaces in {file_path}")

# Iterate over all text files in the directory
for file_name in os.listdir(output_dir):
    if file_name.endswith(".txt"):
        file_path = os.path.join(output_dir, file_name)
        clean_extra_spaces(file_path)


Cleaned extra spaces in summarization_outputs/summary_ratio_50.txt
Cleaned extra spaces in summarization_outputs/summary_ratio_40.txt
Cleaned extra spaces in summarization_outputs/summary_ratio_30.txt


In [18]:
!cp -r /content/summarization_outputs /content/drive/MyDrive